In [ ]:
import sys
import warnings
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import time;
from sklearn import preprocessing
import statsmodels.api as sm
from sklearn import metrics
from sklearn.metrics import accuracy_score
from numpy import mean, absolute
from sklearn.cluster import KMeans
import random

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img



######################################## Data Reading #####################################################################
############################################################################################################


print("------ Now Reading the data ------- Wait..")
print("   ")

print("Normal usage: '28/2/2020 19:00'")
print("Normal usage: '14/2/2020 19:00'")
print("Normal usage: '11/2/2020 22:00'")
print("Normal usage: '2/2/2020 19:00'")
print("Sudden unusual consumption: '29/1/2020 22:00'")
print("Normal usage: '26/1/2020 23:00'")
print("Sudden unusual consumption: '13/1/2020 23:00'")
print("Possible illegal consumption: '5/1/2020 19:00'")
print("Normal usage: '19/12/2019 22:00'")
print("   ")



Chaina_features =pd.read_csv('China1.csv', delimiter=',',encoding = 'unicode_escape') # select columns from 1 to 16 and store it
#Chaina_features.info()
#z1=Chaina_features.describe() # You can check the store data by describing them using the function
#print(Chaina_features.shape)
#Chaina_features = Chaina_features.set_index(Chaina_features.iloc[0:0,0:1036])
print("------ Reading the data ------- Complete..")

#############################################################################################################
############################################################################################################

#############################################################################################################
############################################################################################################
print("------ fetching subset of data ------- wait..")
Chaina_features=Chaina_features.head(3000) #select first 2000 customers
print("------ fetched subset of data ------- Complete..")

#############################################################################################################
############################################################################################################
#############################################################################################################
############################################################################################################
print("------ Imputation--Linear Interpolation method start------- wait..")
###https://www.geeksforgeeks.org/python-pandas-dataframe-interpolate/     ##description use to fil NA or NAN
##limit : Maximum number of consecutive NaNs to fill. Must be greater than 0.
Chaina_features = Chaina_features.interpolate(method='linear')
Chaina_features=Chaina_features.replace(np.NaN, 0)
print("------ Imputation--Linear Interpolation method ------- Complete..")
#############################################################################################################
############################################################################################################
#############################################################################################################
############################################################################################################
#############################################################################################################
############################################################################################################

features= Chaina_features.drop('CONS_NO', axis = 1)
feature_list = list(features.columns)
print("------ Outlier detection and removal 3 sigma method start------- wait..")
from numpy import std
feature=np.array(features)
data_mean, data_std = mean(feature), std(feature)
# identify outliers
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
# identify outliers
outliers = [x for x in feature if (x < lower).any() or ( x > upper).any() ]
print('Identified outliers: %d' % len(outliers))
# remove outliers
outliers_removed = [x for x in feature if (x >= lower).all() and (x <= upper).all()]
print('Non-outlier observations: %d' % len(outliers_removed))

newchaina=pd.DataFrame(outliers_removed, columns=feature_list)
print("------ 3 sigma method ------- Complete..")
print(newchaina.shape)
#############################################################################################################
############################################################################################################
#############################################################################################################
############################################################################################################
print("------ Proposed new sampling method start ------- wait..")
import new_sampling_technique
sampl=new_sampling_technique.getsamples(newchaina)
print(sampl.shape)
print("------ Proposed new sampling method  ------- completed..")




#############################################################################################################
############################################################################################################
#############################################################################################################
############################################################################################################
#############################################################################################################
############################################################################################################

labels = np.array(sampl['FLAG'])#Target  feature
features= sampl.drop('FLAG', axis = 1)
feature_list = list(features.columns)#it store all features except  Reg_capacity price

features_org=features;

print('Data normalization---------------start')
normalized_X=preprocessing.normalize(features)
print('Data normalization---------------Complete')
features = pd.DataFrame(normalized_X, columns=feature_list) # Convert to numpy array
Copy_Features_Ext=features;
print('------ Pre-processing ...Done..')
#############################################################################################################

print('------ Done..')
print('------ Splitting the Data ------- Wait..')
Feature_train, Feature_test, Label_train, Label_test = train_test_split(features, labels, test_size=0.25, random_state=7)
print('Training Features Shape:', Feature_train.shape)
print('Training Labels Shape:', Label_train.shape)
print('Testing Features Shape:', Feature_test.shape)
print('Testing Labels Shape:', Label_test.shape)
print('------ Done..')
print('################################')


#############################################################################################################
############################################################################################################
#############################################################################################################
############################################################################################################



print(features.shape)
print(labels.shape)

'''
results = model.fit(feat_train_lstm1, Label_train, batch_size=32, epochs=50, callbacks=callbacks,\
                    validation_data=(feat_train_lstm1, Label_train))
'''

# create features in 2-d LIKE 128 HEIGHT & 128 width
X = np.zeros((len(features), 128, 128, 1), dtype=np.float32)
y = np.zeros((len(labels), 128, 128, 1), dtype=np.float32)
print(X.shape, y.shape)

from tqdm import tqdm_notebook
from skimage.transform import resize
from skimage.morphology import label

for n, id_ in tqdm_notebook(enumerate(features), total=len(features)):
    # Load images
    img = features
    x_img = img_to_array(img)
    x_img = resize(x_img, (128, 128, 1), mode = 'constant', preserve_range = True)
    # Load masks
    #mask = img_to_array(labels)
    mask = resize(labels, (128, 128, 1), mode = 'constant', preserve_range = True)
    # Save images after scaling in 128 + 128= 256
    ## if before scaled then no need this
    X[n] = x_img
    y[n] = mask
    print("making 2d data and adding -"+str(n))

print(X)


# Split train and valid
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

#train=pd.DataFrame(X_train)
#print(train)
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_valid.shape)
print('Testing Labels Shape:', y_valid.shape)
print('------ Done..')


###########################Proposed Model start###################################
#############################################################################################################
############################################################################################################
#############################################################################################################
##################
                        ###GRU
##############################################################################################################
##################################################################################################################

from keras.layers import LeakyReLU
from numpy import mean, absolute
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

feats_count = Feature_train.shape[1]
samples_count = Feature_train.shape[0]
print(samples_count)
print(feats_count)
Feature_train_lstm = np.expand_dims(Feature_train, axis=2)# new  shape  is  to be added
Feature_test_lstm = np.expand_dims(Feature_test, axis=2)

label_train_lstm = np.expand_dims(Label_train, axis=1)

print(Feature_train_lstm.shape, Label_train.shape)
print(Feature_train_lstm.shape, label_train_lstm.shape)

def create_dnn():
    K.clear_session()
    model = Sequential()

    model.add(GRU(units=30, return_sequences=True, input_shape=(1034, 1)))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    '''Dropout layer is added to avoid over-fitting,
    which is a phenomenon where a machine learning model performs better on the training data
     compared to the test data. Execute the following script to add dropout layer.'''
    #model.add(Dropout(0.2))
    ##Let's add three more GRU and dropout layers to our model. Run the following script.


    model.add(GRU(units=10))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dropout(0.1))

    model.add(Dense(units=1))

    '''To make our model more robust, we add a dense layer at the end of the model.
     The number of neurons in the dense layer will be set to 1 since we want to predict a single value in the output.
    '''
    model.summary()
    model.compile(optimizer = "adam", loss = "binary_crossentropy")
    return model



dnn_model = KerasClassifier(build_fn=create_dnn, batch_size=32, verbose=1)

callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1)]

history=dnn_model.fit(Feature_train_lstm, Label_train, batch_size = 32, epochs=15
              ,validation_data=(Feature_test_lstm, Label_test), callbacks=callbacks)
y_pred1=dnn_model.predict(Feature_test_lstm)

y_pred1=dnn_model.predict(Feature_train_lstm)
y_pred2=dnn_model.predict(Feature_test_lstm)
print(y_pred2)
print(y_pred1)



plt.figure(figsize=(6,8))
plt.title("Learning curve")
plt.plot(history.history["loss"],'-b', label="loss")
plt.plot(history.history["val_loss"],'-r', label="val_loss")
plt.plot( np.argmin(history.history["val_loss"]), np.min(history.history["val_loss"]), marker="x", color="black", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();




############################################################################################################
#####################################################DenseNetFCN modulel########################################################
############################################################################################################

def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    return x

def get_denseNetFCN(input_img, n_filters = 64, dropout = 0.1, batchnorm = True):
    """Function to define the DenseFCN Model"""

    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)

    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)

    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)


    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)

    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)

    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)

    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs], name="DenseNetFCN")
    return model

input_img = Input((128, 128, 1))
model = get_denseNetFCN(input_img, n_filters=16, dropout=0.01, batchnorm=True)
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model-tgs-salt.h5', verbose=1,save_best_only=True, save_weights_only=True)
]


results = model.fit(X_train, y_train, batch_size=32, epochs=15, callbacks=callbacks,\
                    validation_data=(X_valid, y_valid))


plt.figure(figsize=(8, 6))
plt.plot(results.history["loss"], '-b', label="loss")
plt.plot(results.history["val_loss"], '-g', label="val_loss")
plt.plot( np.argmin(results.history["val_loss"]), np.min(results.history["val_loss"]), marker="x", color="black", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

model.load_weights('model-tgs-salt.h5')

# Evaluate on validation set (this must be equals to the best log_loss)
##tells validation loss and accuracy
model.evaluate(X_valid, y_valid, verbose=1)

preds_train = model.predict(X_train, verbose=1)
preds_val = model.predict(X_valid, verbose=1)
densentfcntrain=Feature_train
densentfcntest=Feature_test
print(preds_train.shape, preds_val.shape)



import lightgbm as lgb

joinedtrain=np.concatenate((densentfcntrain,y_pred1), axis=1)
joinedtest=np.concatenate((densentfcntest,y_pred2), axis=1)

classifier = lgb.LGBMClassifier(max_depth = 5,
                         lambda_l1 = 0.1,
                         lambda_l2 = 0.01,
                         learning_rate = 0.01,
                         n_estimators = 200, reg_alpha = 1.1, colsample_bytree = 0.9, subsample = 0.9,
                         n_jobs = 5)
classifier.fit(joinedtrain, Label_train, eval_set=[(joinedtest, Label_test)], eval_metric='accuracy',
               verbose=False, early_stopping_rounds=50);
classifier.fit(joinedtest, Label_test, eval_set=[(joinedtrain, Label_train)], eval_metric='accuracy',
               verbose=False, early_stopping_rounds=50);

predictions = classifier.predict(joinedtest)
#predictions = classifier.predict(joinedtest)
a = accuracy_score(Label_test, classifier.predict(joinedtest))
b = accuracy_score(Label_train, classifier.predict(joinedtrain))

print("Accuracy on test data",a,"Accuracy on train data",b)

#############################################################################################################
################################################Validation############################################################
#############################################################################################################
############################################################################################################

print("Confusion matrix is: ", confusion_matrix(Label_test, predictions))

from sklearn.metrics import f1_score
f1score=f1_score(Label_test, predictions, average='binary')
print("F1score is: ", f1score)


from sklearn.metrics import recall_score
recall = recall_score(Label_test, predictions)
print('Recall score: %f' % recall)

from sklearn.metrics import precision_score
precision = precision_score(Label_test, predictions)
print('Precision: %f' % precision)


# ROC AUC score
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(Label_test, predictions)
print('ROC AUC: %f' % auc)
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve

from sklearn.metrics import matthews_corrcoef
MCC=matthews_corrcoef(Label_test, predictions)
print('MCC: %f' % MCC)



# ROC AUC plot
from sklearn.metrics import roc_curve
fpr1_lr, tpr1_lr, _= roc_curve(Label_test, predictions)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr1_lr, tpr1_lr, label='Proposed model')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()




from sklearn.metrics import average_precision_score
average_precision = average_precision_score(Label_test, predictions)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))


from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
disp = plot_precision_recall_curve(classifier, joinedtest, Label_test, name="Our proposed model")
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))


############################################Proposed model end ########################################

################################## model 1 Wide n Deep CNN   ###################
#############################################################################################################
############################################################################################################
#############################################################################################################
############################################################################################################
                                        ### CNN
print("W&D-CNN ----------------start")
###################################################################################
from keras.models import Sequential      # Model
from keras.layers import Dense            # lAYER
from keras.layers import Dropout
from keras.layers import MaxPooling1D
from keras.layers import Conv1D
from keras.layers import Convolution2D
from keras.layers import Flatten
from keras.callbacks import EarlyStopping
from keras import backend as K


feats_count = Feature_train.shape[1]
samples_count = Feature_test.shape[0]
print(samples_count)
print(feats_count)
Feature_train_CNN = np.expand_dims(Feature_train, axis=2)# new  shape  is  to be added
Feature_test_CNN = np.expand_dims(Feature_test, axis=2)


K.clear_session()
wide = Sequential()

wide.add(Conv1D(32, 3, input_shape=(1034,1)))
wide.add(Activation('relu'))
wide.add(MaxPooling1D(pool_size=(2)))
wide.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
wide.add(Dense(1))
wide.add(Activation('sigmoid'))
wide.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics=['accuracy'])
wide.summary()
early_stop = EarlyStopping(monitor='val_loss', patience=10,min_delta=1)
wide.fit(Feature_train_CNN, Label_train, epochs=15, batch_size=32, verbose=1,validation_split=0.25,callbacks=[early_stop])
wide_pred=wide.predict(Feature_train_CNN);
wide_pred1=wide.predict(Feature_test_CNN);


batch_size = 32
nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 1

K.clear_session()
cnn = Sequential()

cnn.add(Conv2D(32, (3, 3), input_shape=(128,128,1)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Conv2D(32, (3, 3)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Conv2D(64, (3, 3)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

cnn.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors

cnn.add(Dense(1))
cnn.add(Activation('sigmoid'))

cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics=['accuracy'])

cnn.summary()
early_stop = EarlyStopping(monitor='val_loss', patience=10,min_delta=1)
cnn.fit(X_train, Label_train, epochs=10, batch_size=32, verbose=1,validation_split=0.25,callbacks=[early_stop])
cnn_pred=cnn.predict(X_valid);  # Used to predict

#############################################################################################################
wide_pred1=wide_pred1.round()

from sklearn.metrics import f1_score
f1score=f1_score(Label_test, wide_pred1, average='binary')
print("F1score is: ", f1score)


print("Accuracy:",metrics.accuracy_score(Label_test,wide_pred1))

print("Precision:",metrics.precision_score(Label_test, wide_pred1))

print("Recall:",metrics.recall_score(Label_test, wide_pred1))

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(Label_test, wide_pred1)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))



from sklearn.metrics import matthews_corrcoef
MCC=matthews_corrcoef(Label_test, wide_pred1)
print('MCC: %f' % MCC)


# ROC AUC
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(Label_test, wide_pred1)
print('ROC AUC: %f' % auc)

fpr2_lr, tpr2_lr, _= roc_curve(Label_test, wide_pred1)
plt.figure(figsize=(8,6))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr2_lr, tpr2_lr, label='W&D-CNN')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

print("W&D-CNN ----------------end")




################################ Model 2 CNN-LSTM-MLP start #####################################

################################ Model 2 CNN-LSTM-MLP start #####################################
#                        ###Lstm
###############################################################################################################
###################################################################################################################
#
from keras.layers import LeakyReLU
from numpy import mean, absolute
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

feats_count = Feature_train.shape[1]
samples_count = Feature_train.shape[0]
print(samples_count)
print(feats_count)
Feature_train_lstm = np.expand_dims(Feature_train, axis=2)# new  shape  is  to be added
Feature_test_lstm = np.expand_dims(Feature_test, axis=2)

label_train_lstm = np.expand_dims(Label_train, axis=1)

print(Feature_train_lstm.shape, Label_train.shape)
print(Feature_train_lstm.shape, label_train_lstm.shape)

def create_dnn():
    K.clear_session()
    model = Sequential()

    model.add(LSTM(units=20, return_sequences=True, input_shape=(1034, 1)))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    '''Dropout layer is added to avoid over-fitting,
    which is a phenomenon where a machine learning model performs better on the training data
     compared to the test data. Execute the following script to add dropout layer.'''
    #model.add(Dropout(0.2))
    ##Let's add three more GRU and dropout layers to our model. Run the following script.


    model.add(LSTM(units=20))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dropout(0.1))

    model.add(Dense(units=1))

    '''To make our model more robust, we add a dense layer at the end of the model.
     The number of neurons in the dense layer will be set to 1 since we want to predict a single value in the output.
    '''
    model.summary()
    model.compile(optimizer = "adam", loss = "binary_crossentropy")
    return model



dnn_model = KerasClassifier(build_fn=create_dnn, batch_size=32, verbose=1)

callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1)]

history=dnn_model.fit(Feature_train_lstm, Label_train, batch_size = 32, epochs=15
              ,validation_data=(Feature_test_lstm, Label_test), callbacks=callbacks)
y_pred1=dnn_model.predict(Feature_test_lstm)

y_pred1=dnn_model.predict(Feature_train_lstm)
y_pred2=dnn_model.predict(Feature_test_lstm)
print(y_pred2)
print(y_pred1)



plt.figure(figsize=(6,8))
plt.title("Learning curve")
plt.plot(history.history["loss"],'-b', label="loss")
plt.plot(history.history["val_loss"],'-r', label="val_loss")
plt.plot( np.argmin(history.history["val_loss"]), np.min(history.history["val_loss"]), marker="x", color="black", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();


#######################################CNN#############################################
#########################################################################################################



from keras.models import Sequential      # Model
from keras.layers import Dense            # lAYER
from keras.layers import Dropout
from keras.layers import MaxPooling1D
from keras.layers import Conv1D
from keras.layers import Flatten
from keras.callbacks import EarlyStopping
from keras import backend as K


feats_count = Feature_train.shape[1]
samples_count = Feature_train.shape[0]
print(samples_count)
print(feats_count)
Feature_train_CNN = np.expand_dims(Feature_train, axis=2)# new  shape  is  to be added
Feature_test_CNN = np.expand_dims(Feature_test, axis=2)

label_train_CNN = np.expand_dims(Label_train, axis=1)

print(Feature_train_CNN.shape, Label_train.shape)
print(Feature_train_CNN.shape, label_train_CNN.shape)
K.clear_session()
classifier = Sequential()
classifier.add(Conv1D(32, kernel_size=(3), activation='relu', padding='same' ,input_shape=(feats_count,1)))
print('FT:',(len(np.transpose(Feature_train)))); # Units are the neurons in layer
classifier.add(MaxPooling1D(pool_size=2,padding='same'))
classifier.add(Dropout(0.0001));
classifier.add(Flatten())
classifier.add(Dense(2))
classifier.add(Dropout(0.01)); #Dropout is a a technique used to tackle Overfitting and reducre complexity
    #Dropout is a regularization technique, which aims to reduce the complexity of the model with the goal to prevent overfitting.
classifier.add(Dense(1)) # A dense Layer have 1 neuron. it works good in my Scenerio. you can change it
classifier.compile(optimizer = 'adam', loss = "binary_crossentropy" )
    #Compile:Before training a model, you need to configure the learning process, which is done via the compile
    ##loss function: This is the objective that the model will try to minimize. It can be the string identifier of an existing loss function (such as categorical_crossentropy or mse), or it can be an objective function
    ##metrics: For any classification problem you will want to set this to metrics=['accuracy']. A metric could be the string identifier of an existing metric or a custom metric function.
classifier.summary()
early_stop = EarlyStopping(monitor='val_loss', patience=10,min_delta=1)
history = classifier.fit(Feature_train_CNN, Label_train, epochs=100, batch_size=25, verbose=0,validation_split=0.25,callbacks=[early_stop])
predictions_CNN=classifier.predict(Feature_test_CNN);  # Used to predict
predictions_CNN=predictions_CNN;
print(predictions_CNN.shape)





plt.figure(figsize=(6,8))
plt.title("Learning curve")
plt.plot(history.history["loss"],'-b', label="loss")
plt.plot(history.history["val_loss"],'-r', label="val_loss")
plt.plot( np.argmin(history.history["val_loss"]), np.min(history.history["val_loss"]), marker="x", color="black", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();



####################################### MLP #########################################

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
joinedtrain=np.concatenate((Feature_train,y_pred1), axis=1)
joinedtest=np.concatenate((Feature_test,y_pred2), axis=1)

print('MLP Classifier start from here:')
#########################################################################################
classifier=MLPClassifier(solver='lbfgs',alpha=0.0001, verbose=0, max_iter=200)
classifier.fit(joinedtrain, Label_train);
lstmcnnmlp=classifier.predict(joinedtest)
print('MLP Classifier end here:')


print("Confusion matrix is: ", confusion_matrix(Label_test, lstmcnnmlp))

from sklearn.metrics import f1_score
f1score=f1_score(Label_test, lstmcnnmlp, average='binary')
print("F1score is: ", f1score)


from sklearn.metrics import recall_score
recall = recall_score(Label_test, lstmcnnmlp)
print('Recall score: %f' % recall)

from sklearn.metrics import precision_score
precision = precision_score(Label_test, lstmcnnmlp)
print('Precision: %f' % precision)


# ROC AUC score
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(Label_test, lstmcnnmlp)
print('ROC AUC: %f' % auc)
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve

from sklearn.metrics import matthews_corrcoef
MCC=matthews_corrcoef(Label_test, lstmcnnmlp)
print('MCC: %f' % MCC)



# ROC AUC plot
from sklearn.metrics import roc_curve
fpr3_lr, tpr3_lr, _ = roc_curve(Label_test, lstmcnnmlp)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr3_lr, tpr3_lr, label='Proposed model')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()



from sklearn.metrics import average_precision_score

## plot the precision-recall curves
#no_skill = len(Label_test[[predictions]==100]) / len(predictions)
#pyplot.plot([0, 1], [0, 1],[no_skill, no_skill], linestyle='--', label='No Skill')
#pyplot.plot(recall, precision, marker='.', label='Proposed')
## axis labels
#pyplot.xlabel('Recall')
#pyplot.ylabel('Precision')
## show the legend
#pyplot.legend()
## show the plot
#pyplot.show()


average_precision = average_precision_score(Label_test, lstmcnnmlp)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))


from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
disp = plot_precision_recall_curve(classifier, joinedtest, Label_test, name="Our proposed model")
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))



################################ Model 2 CNN-LSTM-MLP end #####################################

############################# Model 3 GRU-MLP start ##############################


                        ###GRU
##############################################################################################################
##################################################################################################################

from keras.layers import LeakyReLU
from numpy import mean, absolute
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

feats_count = Feature_train.shape[1]
samples_count = Feature_train.shape[0]
print(samples_count)
print(feats_count)
Feature_train_lstm = np.expand_dims(Feature_train, axis=2)# new  shape  is  to be added
Feature_test_lstm = np.expand_dims(Feature_test, axis=2)

label_train_lstm = np.expand_dims(Label_train, axis=1)

print(Feature_train_lstm.shape, Label_train.shape)
print(Feature_train_lstm.shape, label_train_lstm.shape)

def create_dnn():
    K.clear_session()
    model = Sequential()

    model.add(GRU(units=30, return_sequences=True, input_shape=(1034, 1)))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    '''Dropout layer is added to avoid over-fitting,
    which is a phenomenon where a machine learning model performs better on the training data
     compared to the test data. Execute the following script to add dropout layer.'''
    #model.add(Dropout(0.2))
    ##Let's add three more GRU and dropout layers to our model. Run the following script.


    model.add(GRU(units=10))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dropout(0.1))

    model.add(Dense(units=1))

    '''To make our model more robust, we add a dense layer at the end of the model.
     The number of neurons in the dense layer will be set to 1 since we want to predict a single value in the output.
    '''
    model.summary()
    model.compile(optimizer = "adam", loss = "binary_crossentropy")
    return model



dnn_model = KerasClassifier(build_fn=create_dnn, batch_size=32, verbose=1)

callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1)]

history=dnn_model.fit(Feature_train_lstm, Label_train, batch_size = 32, epochs=15
              ,validation_data=(Feature_test_lstm, Label_test), callbacks=callbacks)
y_pred1=dnn_model.predict(Feature_test_lstm)

y_pred1=dnn_model.predict(Feature_train_lstm)
y_pred2=dnn_model.predict(Feature_test_lstm)
print(y_pred2)
print(y_pred1)



plt.figure(figsize=(6,8))
plt.title("Learning curve")
plt.plot(history.history["loss"],'-b', label="loss")
plt.plot(history.history["val_loss"],'-r', label="val_loss")
plt.plot( np.argmin(history.history["val_loss"]), np.min(history.history["val_loss"]), marker="x", color="black", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

####################################### MLP #########################################

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
joinedtrain=np.concatenate((Feature_train,y_pred1), axis=1)
joinedtest=np.concatenate((Feature_test,y_pred2), axis=1)

print('MLP Classifier start from here:')
#########################################################################################
classifier=MLPClassifier(solver='lbfgs',alpha=0.0001, verbose=0, max_iter=200)
classifier.fit(joinedtrain, Label_train);
grumlp=classifier.predict(joinedtest)
print('MLP Classifier end here:')


print("Confusion matrix is: ", confusion_matrix(Label_test, grumlp))

from sklearn.metrics import f1_score
f1score=f1_score(Label_test, grumlp, average='binary')
print("F1score is: ", f1score)


from sklearn.metrics import recall_score
recall = recall_score(Label_test, grumlp)
print('Recall score: %f' % recall)

from sklearn.metrics import precision_score
precision = precision_score(Label_test, grumlp)
print('Precision: %f' % precision)


# ROC AUC score
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(Label_test, grumlp)
print('ROC AUC: %f' % auc)
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve

from sklearn.metrics import matthews_corrcoef
MCC=matthews_corrcoef(Label_test, grumlp)
print('MCC: %f' % MCC)



# ROC AUC plot
from sklearn.metrics import roc_curve
fpr4_lr, tpr4_lr, _= roc_curve(Label_test, grumlp)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr4_lr, tpr4_lr, label='Proposed model')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()



# plot the precision-recall curves
no_skill = len(Label_test[Label_test==1]) / len(Label_test)
pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
pyplot.plot(recall, precision, marker='.', label='Proposed')
# axis labels
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()


from sklearn.metrics import average_precision_score
average_precision = average_precision_score(Label_test, grumlp)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))


from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
disp = plot_precision_recall_curve(classifier, joinedtest, Label_test, name="Our proposed model")
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))


############################# Model 3 GRU-MLP end ##############################


############################# Model 4 LSTM-MLP start ##############################



#                        ###Lstm
###############################################################################################################
###################################################################################################################
#
from keras.layers import LeakyReLU
from numpy import mean, absolute
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

feats_count = Feature_train.shape[1]
samples_count = Feature_train.shape[0]
print(samples_count)
print(feats_count)
Feature_train_lstm = np.expand_dims(Feature_train, axis=2)# new  shape  is  to be added
Feature_test_lstm = np.expand_dims(Feature_test, axis=2)

label_train_lstm = np.expand_dims(Label_train, axis=1)

print(Feature_train_lstm.shape, Label_train.shape)
print(Feature_train_lstm.shape, label_train_lstm.shape)

def create_dnn():
    K.clear_session()
    model = Sequential()

    model.add(LSTM(units=20, return_sequences=True, input_shape=(1034, 1)))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    '''Dropout layer is added to avoid over-fitting,
    which is a phenomenon where a machine learning model performs better on the training data
     compared to the test data. Execute the following script to add dropout layer.'''
    #model.add(Dropout(0.2))
    ##Let's add three more GRU and dropout layers to our model. Run the following script.


    model.add(LSTM(units=20))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dropout(0.1))

    model.add(Dense(units=1))

    '''To make our model more robust, we add a dense layer at the end of the model.
     The number of neurons in the dense layer will be set to 1 since we want to predict a single value in the output.
    '''
    model.summary()
    model.compile(optimizer = "adam", loss = "binary_crossentropy")
    return model



dnn_model = KerasClassifier(build_fn=create_dnn, batch_size=32, verbose=1)

callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1)]

history=dnn_model.fit(Feature_train_lstm, Label_train, batch_size = 32, epochs=15
              ,validation_data=(Feature_test_lstm, Label_test), callbacks=callbacks)
y_pred1=dnn_model.predict(Feature_test_lstm)

y_pred1=dnn_model.predict(Feature_train_lstm)
y_pred2=dnn_model.predict(Feature_test_lstm)
print(y_pred2)
print(y_pred1)



plt.figure(figsize=(6,8))
plt.title("Learning curve")
plt.plot(history.history["loss"],'-b', label="loss")
plt.plot(history.history["val_loss"],'-r', label="val_loss")
plt.plot( np.argmin(history.history["val_loss"]), np.min(history.history["val_loss"]), marker="x", color="black", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();



####################################### MLP #########################################

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
joinedtrain=np.concatenate((Feature_train,y_pred1), axis=1)
joinedtest=np.concatenate((Feature_test,y_pred2), axis=1)

print('MLP Classifier start from here:')
#########################################################################################
classifier=MLPClassifier(solver='lbfgs',alpha=0.0001, verbose=0, max_iter=200)
classifier.fit(joinedtrain, Label_train);
lstmmlp=classifier.predict(joinedtest)
print('MLP Classifier end here:')


print("Confusion matrix is: ", confusion_matrix(Label_test, lstmmlp))

from sklearn.metrics import f1_score
f1score=f1_score(Label_test, lstmmlp, average='binary')
print("F1score is: ", f1score)


from sklearn.metrics import recall_score
recall = recall_score(Label_test, lstmmlp)
print('Recall score: %f' % recall)

from sklearn.metrics import precision_score
precision = precision_score(Label_test, lstmmlp)
print('Precision: %f' % precision)


# ROC AUC score
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(Label_test, lstmmlp)
print('ROC AUC: %f' % auc)
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve

from sklearn.metrics import matthews_corrcoef
MCC=matthews_corrcoef(Label_test, lstmmlp)
print('MCC: %f' % MCC)



# ROC AUC plot
from sklearn.metrics import roc_curve
fpr5_lr, tpr5_lr, _ = roc_curve(Label_test, lstmmlp)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr5_lr, tpr5_lr, label='Proposed model')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()



# plot the precision-recall curves
no_skill = len(Label_test[Label_test==1]) / len(Label_test)
pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
pyplot.plot(recall, precision, marker='.', label='Proposed')
# axis labels
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()


from sklearn.metrics import average_precision_score
average_precision = average_precision_score(Label_test, lstmmlp)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))


from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
disp = plot_precision_recall_curve(classifier, joinedtest, Label_test, name="Our proposed model")
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(average_precision))




############################# Model 4 LSTM-MLP END ##############################



from sklearn.metrics import roc_curve
fpr1_lr, tpr1_lr, _ = roc_curve(Label_test, predictions)
fpr2_lr, tpr2_lr, _ = roc_curve(Label_test, wide_pred1)
fpr3_lr, tpr3_lr, _ = roc_curve(Label_test, lstmcnnmlp)
fpr4_lr, tpr4_lr, _ = roc_curve(Label_test, grumlp)
fpr5_lr, tpr5_lr, _ = roc_curve(Label_test, lstmmlp)
plt.figure(figsize=(8,6))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr1_lr, tpr1_lr,'b', label='Proposed model')
plt.plot(fpr2_lr, tpr2_lr,'#8c564b', label='W&D-CNN')
plt.plot(fpr3_lr, tpr3_lr,'r', label='CNN_LSTM-MLP')
plt.plot(fpr4_lr, tpr4_lr,'#1f77b4', label='GRU-MLP')
plt.plot(fpr5_lr, tpr5_lr,'#d62728', label='LSTM-MLP')
plt.xlabel('False positive rate', fontsize=14)
plt.ylabel('True positive rate', fontsize=14)
plt.legend(loc='best')
plt.show()

------ Now Reading the data ------- Wait..
   
Normal usage: '28/2/2020 19:00'
Normal usage: '14/2/2020 19:00'
Normal usage: '11/2/2020 22:00'
Normal usage: '2/2/2020 19:00'
Sudden unusual consumption: '29/1/2020 22:00'
Normal usage: '26/1/2020 23:00'
Sudden unusual consumption: '13/1/2020 23:00'
Possible illegal consumption: '5/1/2020 19:00'
Normal usage: '19/12/2019 22:00'
   
------ Reading the data ------- Complete..
------ fetching subset of data ------- wait..
------ fetched subset of data ------- Complete..
------ Imputation--Linear Interpolation method start------- wait..
------ Imputation--Linear Interpolation method ------- Complete..
------ Outlier detection and removal 3 sigma method start------- wait..
Identified outliers: 23
Non-outlier observations: 2977
------ 3 sigma method ------- Complete..
(2977, 1035)
------ Proposed new sampling method start ------- wait..
(2977, 1035)
theftysamples are: 514
honestysamples are: 2463
their difference is: 1949
their shapes are: (514

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:181: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


making 2d data and adding -0
making 2d data and adding -1
making 2d data and adding -2
making 2d data and adding -3
making 2d data and adding -4
making 2d data and adding -5
making 2d data and adding -6
making 2d data and adding -7
making 2d data and adding -8
making 2d data and adding -9
making 2d data and adding -10
making 2d data and adding -11
making 2d data and adding -12
making 2d data and adding -13
making 2d data and adding -14
making 2d data and adding -15
making 2d data and adding -16
making 2d data and adding -17
making 2d data and adding -18
making 2d data and adding -19
making 2d data and adding -20
making 2d data and adding -21
making 2d data and adding -22
making 2d data and adding -23
making 2d data and adding -24
making 2d data and adding -25
making 2d data and adding -26
making 2d data and adding -27
making 2d data and adding -28
making 2d data and adding -29
making 2d data and adding -30
making 2d data and adding -31
making 2d data and adding -32
making 2d data and a